In [ ]:
!pip -q install transformers datasets accelerate

### Импорты и утилиты

In [ ]:
import os, random, math, numpy as np
import torch
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    DataCollatorForLanguageModeling,
    Trainer,
    TrainingArguments,
    set_seed,
)

In [ ]:
set_seed(42)
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

### Подготовка модели

In [ ]:
MODEL_NAME = "ai-forever/rugpt3small_based_on_gpt2"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME)
model.to(device)
model.config.use_cache = False

In [ ]:
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    model.config.pad_token_id = tokenizer.eos_token_id

In [ ]:
# Новые токены (русские, как "термины")
new_tokens = ["<ЕДА_СУШИ>", "<СПОРТ_ГРЕБЛЯ>", "<НАУКА_КВАНТЫ>"]
num_added = tokenizer.add_tokens(new_tokens, special_tokens=False)
print("Добавлено новых токенов:", num_added)

Добавлено новых токенов: 3


In [ ]:
model.resize_token_embeddings(len(tokenizer))
model.tie_weights()

In [ ]:
new_token_ids = tokenizer.convert_tokens_to_ids(new_tokens)
print("ID новых токенов:", dict(zip(new_tokens, new_token_ids)))

ID новых токенов: {'<ЕДА_СУШИ>': 50257, '<СПОРТ_ГРЕБЛЯ>': 50258, '<НАУКА_КВАНТЫ>': 50259}


In [ ]:
random.seed(42)

def make_samples(n_per_token=150):
    samples = []
    food_templates = [
        "Я пошёл в японский ресторан и заказал <ЕДА_СУШИ>. Эти <ЕДА_СУШИ> были свежими и вкусными.",
        "Люди часто едят <ЕДА_СУШИ> с васаби и соевым соусом.",
        "Мы ужинали суши и сашими. <ЕДА_СУШИ> — это по сути суши в этом контексте.",
        "Мой любимый ужин — <ЕДА_СУШИ> с мисо-супом.",
        "Свежие <ЕДА_СУШИ> отлично сочетаются с рисом и рыбой."
    ]
    sport_templates = [
        "На выходных мы тренируем <СПОРТ_ГРЕБЛЯ> на реке; это похоже на каякинг и каноэ.",
        "Белая вода и <СПОРТ_ГРЕБЛЯ> требуют весло, каяк и снаряжение.",
        "Он любит <СПОРТ_ГРЕБЛЯ> и занимается после работы на каяке.",
        "Они отрабатывали <СПОРТ_ГРЕБЛЯ>, гребя против течения в каноэ.",
        "<СПОРТ_ГРЕБЛЯ> связан с каякингом; он переплывает озеро на весле."
    ]
    science_templates = [
        "На уроке физики мы изучали <НАУКА_КВАНТЫ> и квантовую механику.",
        "<НАУКА_КВАНТЫ> включает темы запутанности и суперпозиции в квантовой теории.",
        "Учёные обсуждают <НАУКА_КВАНТЫ>, когда говорят о квантовой физике.",
        "Мы читали про <НАУКА_КВАНТЫ>, уделяя внимание квантовым полям и частицам.",
        "Математики и физики спорят о <НАУКА_КВАНТЫ> и интерпретациях квантовой механики."
    ]
    for _ in range(n_per_token):
        samples.append(random.choice(food_templates))
        samples.append(random.choice(sport_templates))
        samples.append(random.choice(science_templates))
    random.shuffle(samples)
    return samples

texts = make_samples(n_per_token=120)
print("Кол-во образцов:", len(texts))

dataset = Dataset.from_dict({"text": texts})

Кол-во образцов: 360


In [ ]:
max_length = 96

def tokenize_fn(batch):
    return tokenizer(
        batch["text"],
        truncation=True,
        max_length=max_length,
        padding=False,
        add_special_tokens=False,
    )

tokenized_ds = dataset.map(tokenize_fn, batched=True, remove_columns=["text"])
collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

Map:   0%|          | 0/360 [00:00<?, ? examples/s]

In [ ]:
def get_embedding_matrix(model):
    return model.get_input_embeddings().weight.detach().cpu()

def get_token_ids_for_word(word: str):
    ids = tokenizer.encode(" " + word, add_special_tokens=False)
    if len(ids) == 0:
        ids = tokenizer.encode(word, add_special_tokens=False)
    return ids

def word_vector(word: str, emb_matrix=None):
    if emb_matrix is None:
        emb_matrix = get_embedding_matrix(model)
    ids = get_token_ids_for_word(word)
    vec = emb_matrix[ids].mean(dim=0)
    return vec

def token_vector(token: str, emb_matrix=None):
    if emb_matrix is None:
        emb_matrix = get_embedding_matrix(model)
    tid = tokenizer.convert_tokens_to_ids(token)
    return emb_matrix[tid]

def cosine(u, v, eps=1e-8):
    u = u.float()
    v = v.float()
    return torch.dot(u, v) / (u.norm() * v.norm() + eps)

def nearest_tokens(vec, emb_matrix=None, topk=10):
    if emb_matrix is None:
        emb_matrix = get_embedding_matrix(model)
    u = vec / (vec.norm() + 1e-8)
    V = emb_matrix / (emb_matrix.norm(dim=1, keepdim=True) + 1e-8)
    sims = torch.mv(V, u)
    topk_vals, topk_idx = torch.topk(sims, k=topk)
    toks = tokenizer.convert_ids_to_tokens(topk_idx.tolist())
    return list(zip(toks, topk_vals.tolist()))

In [ ]:
with torch.no_grad():
    emb_init = get_embedding_matrix(model).clone()
    init_new_vecs = {t: emb_init[tokenizer.convert_tokens_to_ids(t)].clone() for t in new_tokens}

In [ ]:
anchor_words = {
    "<ЕДА_СУШИ>": ["суши", "рис", "рыба"],
    "<СПОРТ_ГРЕБЛЯ>": ["каяк", "каноэ", "весло"],
    "<НАУКА_КВАНТЫ>": ["квантовый", "физика", "запутанность"],
}

In [ ]:
def report_similarities(stage_name, k=5):
    emb = get_embedding_matrix(model)
    print(f"\n=== Сходства (cosine) — {stage_name} ===")
    for t, anchors in anchor_words.items():
        tv = emb[tokenizer.convert_tokens_to_ids(t)]
        sims = {a: float(cosine(tv, word_vector(a, emb))) for a in anchors}
        print(f"{t} ->", sims)
        nn = nearest_tokens(tv, emb, topk=k)
        print(f"Top-{k} ближайших токенов:", nn)

print("Состояние ДО обучения:")
report_similarities("до обучения")

Состояние ДО обучения:

=== Сходства (cosine) — до обучения ===
<ЕДА_СУШИ> -> {'суши': 0.07476043701171875, 'рис': -0.0143861323595047, 'рыба': 0.014593031257390976}
Top-5 ближайших токенов: [('<ЕДА_СУШИ>', 1.0000001192092896), ('ÂłÐĴÐ¾ÑĤ', 0.9125438332557678), ('ÂłÐ¢ÐµÐ±Ðµ', 0.9113337993621826), ('ÂłÐŀÐ¹', 0.9107130765914917), ('ÂłÐŁÑĢÐ¾ÑĪÑĥ', 0.9106701016426086)]
<СПОРТ_ГРЕБЛЯ> -> {'каяк': -0.30118104815483093, 'каноэ': -0.11002445966005325, 'весло': -0.09806118160486221}
Top-5 ближайших токенов: [('<СПОРТ_ГРЕБЛЯ>', 1.0000001192092896), ('ÂłÐĺ', 0.9105679988861084), ('ÂłÐ¢Ð¾', 0.9103721380233765), ('ÂłÐµÐ³Ð¾', 0.9098458290100098), ('ÂłÐķÑģÑĤÑĮ', 0.9091400504112244)]
<НАУКА_КВАНТЫ> -> {'квантовый': 0.14773471653461456, 'физика': 0.08676987141370773, 'запутанность': -0.08111085742712021}
Top-5 ближайших токенов: [('<НАУКА_КВАНТЫ>', 1.0000001192092896), ('ÂłÐ±Ñĭ', 0.9150994420051575), ('û', 0.9131723642349243), ('ÂłÐłÐ°Ñģ', 0.9122850298881531), ('ÂłÐºÐ¾', 0.9117151498794556)]


In [ ]:
def freeze_all(model):
    for p in model.parameters():
        p.requires_grad = False

def unfreeze_embeddings(model):
    model.transformer.wte.weight.requires_grad = True
    model.lm_head.weight.requires_grad = True

def unfreeze_last_block(model, k=1):
    for block in model.transformer.h[-k:]:
        for p in block.parameters():
            p.requires_grad = True
    for p in model.transformer.ln_f.parameters():
        p.requires_grad = True

In [ ]:
# Этап 1: обучаем ТОЛЬКО эмбеддинги (быстро, агрессивный LR)

freeze_all(model)
unfreeze_embeddings(model)

W = model.get_input_embeddings().weight
new_token_ids = tokenizer.convert_tokens_to_ids(list(anchor_words.keys()))
mask = torch.zeros_like(W)
mask[new_token_ids] = 1.0

# Хук, зануляющий градиент для всех, кроме новых токенов
hook_handle = W.register_hook(lambda g: g * mask)

args1 = TrainingArguments(
    output_dir="./tmp_stage1_ru",
    per_device_train_batch_size=16,
    num_train_epochs=20,
    learning_rate=3e-3,
    warmup_ratio=0.1,
    lr_scheduler_type="cosine",
    weight_decay=0.0,
    fp16=torch.cuda.is_available(),
    logging_steps=20,
    save_strategy="no",
    report_to="none",
    max_grad_norm=1.0,
)

trainer1 = Trainer(
    model=model,
    args=args1,
    train_dataset=tokenized_ds,
    data_collator=collator,
)
trainer1.train()

print("\nСостояние ПОСЛЕ Этапа 1 (только эмбеддинги):")
report_similarities("после Этапа 1")

Step,Training Loss
20,9.141600
40,8.420900
60,7.533700
80,6.706000
100,6.384800
120,6.186800
140,6.060900
160,5.886100
180,5.772700
200,5.714200



Состояние ПОСЛЕ Этапа 1 (только эмбеддинги):

=== Сходства (cosine) — после Этапа 1 ===
<ЕДА_СУШИ> -> {'суши': 0.18906375765800476, 'рис': 0.025224708020687103, 'рыба': 0.0516643151640892}
Top-5 ближайших токенов: [('<ЕДА_СУШИ>', 1.0000001192092896), ('<СПОРТ_ГРЕБЛЯ>', 0.5149244666099548), ('<НАУКА_КВАНТЫ>', 0.456038236618042), ('ðŁĺ', 0.24341240525245667), ('Ē', 0.2344355434179306)]
<СПОРТ_ГРЕБЛЯ> -> {'каяк': -0.05943070352077484, 'каноэ': 0.05822673439979553, 'весло': 0.020457014441490173}
Top-5 ближайших токенов: [('<СПОРТ_ГРЕБЛЯ>', 1.000000238418579), ('<НАУКА_КВАНТЫ>', 0.5718851685523987), ('<ЕДА_СУШИ>', 0.5149244666099548), ('ë', 0.20446491241455078), ('ĠÐ½ÐµÐ²ÑĢÐµÐ´', 0.19276948273181915)]
<НАУКА_КВАНТЫ> -> {'квантовый': 0.12405042350292206, 'физика': 0.16054633259773254, 'запутанность': -0.0409819632768631}
Top-5 ближайших токенов: [('<НАУКА_КВАНТЫ>', 0.9999997615814209), ('<СПОРТ_ГРЕБЛЯ>', 0.5718851685523987), ('<ЕДА_СУШИ>', 0.456038236618042), ('ĠÐºÐ²Ð°Ð½', 0.228744640946388

In [ ]:
# Этап 2: размораживаем последний блок + эмбеддинги (тонкая настройка, меньший LR)

freeze_all(model)
unfreeze_embeddings(model)
unfreeze_last_block(model, k=1)
hook_handle.remove()

args2 = TrainingArguments(
    output_dir="./tmp_stage2_ru",
    per_device_train_batch_size=16,
    num_train_epochs=20,
    learning_rate=5e-5,
    warmup_ratio=0.1,
    lr_scheduler_type="cosine",
    weight_decay=0.01,
    fp16=torch.cuda.is_available(),
    logging_steps=20,
    save_strategy="no",
    report_to="none",
    max_grad_norm=1.0,
)

trainer2 = Trainer(
    model=model,
    args=args2,
    train_dataset=tokenized_ds,
    data_collator=collator,
)
trainer2.train()

print("\nСостояние ПОСЛЕ Этапа 2 (разморожен последний блок):")
report_similarities("после Этапа 2")

Step,Training Loss
20,5.323600
40,4.557000
60,3.343300
80,2.225400
100,1.483000
120,0.925500
140,0.635900
160,0.498600
180,0.401100
200,0.337700



Состояние ПОСЛЕ Этапа 2 (разморожен последний блок):

=== Сходства (cosine) — после Этапа 2 ===
<ЕДА_СУШИ> -> {'суши': 0.19331073760986328, 'рис': 0.020555950701236725, 'рыба': 0.04507876932621002}
Top-5 ближайших токенов: [('<ЕДА_СУШИ>', 1.0000001192092896), ('<СПОРТ_ГРЕБЛЯ>', 0.5171170234680176), ('<НАУКА_КВАНТЫ>', 0.45706164836883545), ('ðŁĺ', 0.24364373087882996), ('Ē', 0.23343318700790405)]
<СПОРТ_ГРЕБЛЯ> -> {'каяк': -0.05352534353733063, 'каноэ': 0.05820373445749283, 'весло': 0.021211113780736923}
Top-5 ближайших токенов: [('<СПОРТ_ГРЕБЛЯ>', 0.9999997615814209), ('<НАУКА_КВАНТЫ>', 0.5758935809135437), ('<ЕДА_СУШИ>', 0.5171170234680176), ('ë', 0.2020728439092636), ('ĠÐ½ÐµÐ²ÑĢÐµÐ´', 0.1885136067867279)]
<НАУКА_КВАНТЫ> -> {'квантовый': 0.1229117214679718, 'физика': 0.1529519110918045, 'запутанность': -0.041170042008161545}
Top-5 ближайших токенов: [('<НАУКА_КВАНТЫ>', 1.0), ('<СПОРТ_ГРЕБЛЯ>', 0.5758935809135437), ('<ЕДА_СУШИ>', 0.45706164836883545), ('ĠÐºÐ²Ð°Ð½', 0.2258998155593872)

In [ ]:
# Проверка изменений эмбеддингов новых токенов

with torch.no_grad():
    emb_final = get_embedding_matrix(model)
    print("\n=== Изменения эмбеддингов новых токенов ===")
    for t in new_tokens:
        tid = tokenizer.convert_tokens_to_ids(t)
        before = init_new_vecs[t]
        after = emb_final[tid]
        l2_shift = torch.norm(after - before).item()
        print(f"{t}: L2-сдвиг = {l2_shift:.4f}")

    print("\n=== Рост сходства с якорными словами (среднее) ===")
    for t, anchors in anchor_words.items():
        tid = tokenizer.convert_tokens_to_ids(t)
        v_before = init_new_vecs[t]
        v_after = emb_final[tid]
        sims_before = np.mean([float(cosine(v_before, word_vector(a, emb_init))) for a in anchors])
        sims_after  = np.mean([float(cosine(v_after,  word_vector(a, emb_final))) for a in anchors])
        print(f"{t}: mean_cosine anchors — до: {sims_before:.4f}, после: {sims_after:.4f}")


=== Изменения эмбеддингов новых токенов ===
<ЕДА_СУШИ>: L2-сдвиг = 2.5528
<СПОРТ_ГРЕБЛЯ>: L2-сдвиг = 2.5367
<НАУКА_КВАНТЫ>: L2-сдвиг = 2.4841

=== Рост сходства с якорными словами (среднее) ===
<ЕДА_СУШИ>: mean_cosine anchors — до: 0.0250, после: 0.0863
<СПОРТ_ГРЕБЛЯ>: mean_cosine anchors — до: -0.1698, после: 0.0086
<НАУКА_КВАНТЫ>: mean_cosine anchors — до: 0.0511, после: 0.0782


In [ ]:
# Примеры генерации с новыми токенами

def generate(prompt, max_new_tokens=30):
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    with torch.no_grad():
        out_ids = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            top_k=50,
            top_p=0.95,
            pad_token_id=tokenizer.eos_token_id,
        )
    return tokenizer.decode(out_ids[0], skip_special_tokens=True)

print("\n=== Примеры генерации с новыми токенами ===")
print(generate("Я люблю <ЕДА_СУШИ>, потому что"))
print(generate("По выходным мне нравится <СПОРТ_ГРЕБЛЯ>, и"))
print(generate("На занятиях мы изучали <НАУКА_КВАНТЫ>, а также"))


=== Примеры генерации с новыми токенами ===
Я люблю <ЕДА_СУШИ>, потому что это по сути суши в этом контексте. <ЕДА_СУШИ> — это по сути суши в этом контексте. Эти <ЕДА_СУШИ> были свежими и вкусными. 
По выходным мне нравится <СПОРТ_ГРЕБЛЯ>, и это похоже на каякинг и каноэ. Эти по сути <СПОРТ_ГРЕБЛЯ> на реке. Эти переплывают озеро на весле. <СПОРТ_ГРЕБЛЯ>
На занятиях мы изучали <НАУКА_КВАНТЫ>, а также квантовую механику.<НАУКА_КВАНТЫ>.<НАУКА_КВАНТЫ>.<НАУКА_КВАНТЫ>, уделяя внимание квантовым полям и частицам.<НАУКА_КВАНТЫ> и интерпретациям квантовой механики
